In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCS_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(500000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.7)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.05)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(-0.002535)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.002535)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.03)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.068)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.086)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.02)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(3)

    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(3)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2021-05-05 01:44:25 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-05-05 01:44:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
#                 "fs.properties.PR_kappa['R32', 'R125']",
#                  "fs.properties.PR_kappa['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                 "fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-7

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )

In [9]:
solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-05-05 01:44:27 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-05-05 01:44:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-05-05 01:44:30 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-05-05 01:44:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-05-05 01:44:33 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-05-05 01:44:35 [INFO] idaes.init.fs.state_

  36  9.7691883e+03 3.38e+00 6.51e+03  -1.0 5.40e+04    -  4.12e-01 1.87e-04h  1
  37  9.6928632e+03 3.03e+00 3.97e+03  -1.0 6.35e+04    -  6.04e-01 1.87e-01f  1
  38  9.6248057e+03 2.77e+00 4.21e+03  -1.0 5.60e+04    -  7.11e-01 8.82e-02f  1
  39  9.6199379e+03 2.76e+00 5.83e+03  -1.0 4.94e+04    -  7.79e-02 2.16e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  8.4109787e+03 2.58e+00 4.67e+03  -1.0 5.26e+04    -  1.48e-01 5.27e-01f  1
  41  7.7731020e+03 2.27e+00 3.89e+03  -1.0 5.29e+04    -  1.64e-01 2.82e-01f  1
  42  7.7551921e+03 2.25e+00 3.89e+03  -1.0 5.67e+04    -  3.69e-01 7.66e-03f  1
  43  7.6634842e+03 2.19e+00 6.46e+04  -1.0 9.22e+04    -  2.86e-01 2.78e-02f  1
  44  6.3263157e+03 1.80e+00 2.59e+04  -1.0 1.09e+05    -  2.02e-01 3.53e-01f  1
  45  6.2773565e+03 1.78e+00 8.99e+04  -1.0 9.71e+04    -  5.47e-01 1.37e-02f  1
  46  2.8367385e+03 1.15e+00 5.52e+05  -1.0 2.04e+05    -  2.08e-01 8.04e-01f  1
  47  1.9262039e+03 2.75e+00

 127  2.1419468e+00 8.41e-09 2.10e-04  -5.7 1.64e-01  -3.6 1.00e+00 1.00e+00h  1
 128  2.1419132e+00 7.56e-08 4.52e-05  -5.7 4.91e-01  -4.0 1.00e+00 1.00e+00h  1
 129  2.1418126e+00 6.79e-07 5.90e-05  -5.7 1.47e+00  -4.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 130  2.1415104e+00 6.06e-06 8.54e-05  -5.7 4.40e+00  -5.0 1.00e+00 1.00e+00h  1
 131  2.1406008e+00 5.31e-05 5.42e-04  -5.7 1.31e+01  -5.5 1.00e+00 1.00e+00h  1
 132  2.1378376e+00 4.44e-04 2.63e-03  -5.7 3.87e+01  -5.9 1.00e+00 1.00e+00h  1
 133  2.1289920e+00 3.17e-03 1.81e-02  -5.7 1.10e+02  -6.4 1.00e+00 1.00e+00h  1
 134  2.0704841e+00 6.87e-03 6.49e-01  -5.7 4.33e+02  -6.9 1.00e+00 9.38e-01H  1
 135  2.0653990e+00 3.42e-02 5.73e-01  -5.7 1.83e+03    -  1.76e-01 1.25e-01f  4
 136  2.0583808e+00 3.28e-02 2.78e+00  -5.7 5.14e+02  -7.4 1.00e+00 1.57e-01h  1
 137  2.0518588e+00 3.82e-02 3.04e+00  -5.7 1.52e+03    -  3.80e-01 1.25e-01h  4
 138  2.0503003e+00 4.16e-02

 218  6.3993182e-01 5.03e-01 4.85e+05  -8.6 1.81e+02  -6.5 9.99e-03 1.00e+00h  1
 219  6.3823865e-01 1.38e-01 2.33e+01  -8.6 3.78e+01  -6.1 9.73e-02 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 220  6.3762536e-01 8.70e-03 1.64e+00  -8.6 1.40e+01  -5.7 5.14e-01 1.00e+00h  1
 221  6.3734977e-01 5.34e-04 6.75e-04  -8.6 5.25e+00  -5.2 1.00e+00 1.00e+00h  1
 222  6.3733496e-01 3.79e-06 3.09e-05  -8.6 2.46e-01  -3.9 9.69e-01 1.00e+00h  1
 223  6.3729125e-01 3.64e-04 3.22e-04  -8.6 7.38e-01  -4.4 1.00e+00 1.00e+00h  1
 224  6.3729265e-01 3.64e-04 3.18e-04  -8.6 1.11e+01  -3.0 1.46e-02 4.88e-04h 12
 225  6.3728819e-01 4.26e-05 7.30e-05  -8.6 8.38e-02  -2.6 1.00e+00 1.00e+00h  1
 226  6.3726071e-01 1.08e-04 1.17e-04  -8.6 1.34e-01  -3.1 1.00e+00 1.00e+00h  1
 227  6.3717548e-01 4.62e-03 2.54e-02  -8.6 5.36e-01  -3.6 1.00e+00 6.30e-01h  1
 228  6.3667439e-01 3.02e-04 8.60e+01  -8.6 1.94e+01  -4.1 6.87e-08 1.93e-01F  1
 229  6.3647012e-01 1.18e-05

 306  3.3222860e-16 1.27e-01 2.64e-06  -8.6 1.10e-01  -7.8 1.00e+00 1.00e+00h  1
 307  1.4844684e-16 3.78e-02 2.37e-06  -8.6 7.32e-02  -8.2 1.00e+00 1.00e+00h  1
 308  1.4479381e-16 6.95e-02 1.03e-05  -8.6 1.52e-01  -8.7 1.00e+00 1.00e+00h  1
 309  1.6877301e-16 6.52e-02 1.77e-04  -8.6 6.15e-01  -9.2 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310  2.8308501e-16 1.26e-01 3.47e-02  -8.6 2.46e+00  -9.7 1.00e+00 5.00e-01h  2
 311  3.0739597e-16 5.54e-03 1.72e-04  -8.6 2.79e-01  -8.3 1.00e+00 1.00e+00h  1
 312  3.2272996e-16 4.57e-04 4.70e-05  -8.6 1.20e-01  -7.9 1.00e+00 1.00e+00h  1
 313  4.0653533e-16 8.92e-02 1.17e-03  -8.6 7.96e-01  -8.4 1.00e+00 1.00e+00h  1
 314  2.3557260e-15 3.93e-02 3.76e-05  -8.6 4.66e+00    -  1.00e+00 1.00e+00h  1
 315  3.0100324e-15 7.28e+00 2.27e-03  -8.6 1.98e+01    -  1.85e-01 1.00e+00h  1
 316  2.9302052e-15 6.49e+00 2.91e-02  -8.6 1.77e+01    -  3.36e-01 1.09e-01h  2
 317  2.4283364e-15 9.68e-01

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.000000

The values for the parameters are as follows:
fs.properties.PR_kappa[R125,bmimPF6] = 0.045384748521597996
fs.properties.PR_kappa[R32,bmimPF6] = -0.05059689809083274
fs.properties.PR_kappa[bmimPF6,R125] = 1.5338943883690541
fs.properties.PR_kappa[bmimPF6,R32] = 0.3293449628167168


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
